In [2]:
import pandas as pd

# Load the original dataset
df = pd.read_csv(r"..\data\diabetic_data.csv")

# Standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_")

# Drop columns with too many missing values or little relevance
df_clean = df.drop(columns=["payer_code", "medical_specialty"])




In [3]:
# Save the cleaned full dataset
df_clean.to_csv(r"..\data\clean_diabetic_data.csv", index=False)

# Create a simplified patients table
patients = df_clean[["patient_nbr", "race", "gender", "age"]].drop_duplicates()
patients.to_csv(r"..\data\patients.csv", index=False)

# Create a hospital visits table without patient demographic columns
hospital_visits = df_clean.drop(columns=["race", "gender", "age"])
hospital_visits.to_csv(r"..\data\hospital_visits.csv", index=False)

# Preview number of rows for each
len(df_clean), len(patients), len(hospital_visits)

(101766, 73326, 101766)

In [2]:
# Generate SQL create table statements for PostgreSQL

patients = """
CREATE TABLE patients (
    patient_nbr BIGINT PRIMARY KEY,
    race TEXT,
    gender TEXT,
    age TEXT
);
"""

hospital_visits = """
CREATE TABLE hospital_visits (
    encounter_id BIGINT PRIMARY KEY,
    patient_nbr BIGINT REFERENCES patients(patient_nbr),
    admission_type_id INTEGER,
    discharge_disposition_id INTEGER,
    admission_source_id INTEGER,
    time_in_hospital INTEGER,
    num_lab_procedures INTEGER,
    num_procedures INTEGER,
    num_medications INTEGER,
    number_outpatient INTEGER,
    number_emergency INTEGER,
    number_inpatient INTEGER,
    diag_1 TEXT,
    diag_2 TEXT,
    diag_3 TEXT,
    number_diagnoses INTEGER,
    max_glu_serum TEXT,
    a1cresult TEXT,
    metformin TEXT,
    insulin TEXT,
    change TEXT,
    diabetesmed TEXT,
    readmitted TEXT
);
"""

# Save these to .sql files
with open(r"K:\data science\Hospital_analytics_sql\data\create_tables.sql", "w") as f:
    f.write(patients)
    f.write("\n")
    f.write(hospital_visits)

r"K:\data science\Hospital_analytics_sql\data\create_tables.sql"


'K:\\data science\\Hospital_analytics_sql\\data\\create_tables.sql'

In [6]:
with open(r"..\sql\insert_patients.sql", "w") as f:
    for _, row in patients.iterrows():
        query = (
            f"MERGE INTO patients_sql AS target\n"
            f"USING (SELECT {row['patient_nbr']} AS patient_nbr, "
            f"'{row['race']}' AS race, '{row['gender']}' AS gender, '{row['age']}' AS age) AS source\n"
            f"ON target.patient_nbr = source.patient_nbr\n"
            f"WHEN MATCHED THEN\n"
            f"    UPDATE SET race = source.race, gender = source.gender, age = source.age\n"
            f"WHEN NOT MATCHED THEN\n"
            f"    INSERT (patient_nbr, race, gender, age)\n"
            f"    VALUES (source.patient_nbr, source.race, source.gender, source.age);\n\n"
        )
        f.write(query)
